Need to park this notebook somewhere before 05

In [6]:
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env_v02.py
print(config.n)
exec(config.n)


BUCKET_NAME          = "fraud123-438914-fraudfinder"
PROJECT              = "fraud123-438914"
REGION               = "us-central1"
ID                   = "fipp8"
FEATURESTORE_ID      = "fraudfinder_fipp8"
MODEL_NAME           = "ff_model"
ENDPOINT_NAME        = "ff_model_endpoint"
TRAINING_DS_SIZE     = "1000"
DATA_DIR             = "data"
TRAIN_DATA_DIR       = "train"
CUSTOMER_ENTITY      = "customer"
TERMINAL_ENTITY      = "terminal"
TARGET               = "tx_fraud"



In [8]:
#TEMP TO BE DELETED
ID = "fvde2"

In [4]:
# Replace Region here
REGION = "us-central1"

# static parameters
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
UPLOAD_BUCKET = f"model-upload-{PROJECT_ID}"
AGENT_BUCKET = f"ai-workshops-{ID}"
TRAINING_DS_SIZE = 1000

In [9]:
# Gets the default BUCKET_URI and SERVICE_ACCOUNT if they were not specified by the user.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)

Using this default Service Account: 520607199607-compute@developer.gserviceaccount.com


In [28]:
config = f"""
PROJECT_ID: {PROJECT_ID}
BUCKET_NAME: {BUCKET_NAME}
REGION: {REGION}
ID: "{ID}"
CUSTOMER_ENTITY_ID: customer
CUSTOMER_ENTITY_ID_FIELD: customer_id
TERMINAL_ENTITY_ID: terminal
TERMINALS_ENTITY_ID_FIELD: terminal_id
FEATURESTORE_ID: "fraudfinder_{ID}"
FEATUREVIEW_ID: "fraudfinder_view_{ID}"
NETWORK: fraud-finder-network
SUBNET: https://www.googleapis.com/compute/v1/projects/fraud-finder-lab/regions/us-central1/subnetworks/us-central1
MODEL_REGISTRY: ff_model
RAW_BQ_TRANSACTION_TABLE_URI: "{PROJECT_ID}.tx.tx"
RAW_BQ_LABELS_TABLE_URI: "{PROJECT_ID}.tx.txlabels"
FEATURES_BQ_TABLE_URI: "{PROJECT_ID}.tx.wide_features_table"
FEATURE_TIME: feature_ts
ONLINE_STORAGE_NODES: 1
SUBSCRIPTION_NAME: ff-tx-for-feat-eng-sub
SUBSCRIPTION_PATH: "projects/{PROJECT_ID}/subscriptions/ff-tx-for-feat-eng-sub"
DROP_COLUMNS:
- timestamp
- entity_type_customer
- entity_type_terminal
FEAT_COLUMNS:
- customer_id_avg_amount_14day_window
- customer_id_avg_amount_15min_window
- customer_id_avg_amount_1day_window
- customer_id_avg_amount_30min_window
- customer_id_avg_amount_60min_window
- customer_id_avg_amount_7day_window
- customer_id_nb_tx_14day_window
- customer_id_nb_tx_15min_window
- customer_id_nb_tx_1day_window
- customer_id_nb_tx_30min_window
- customer_id_nb_tx_60min_window
- customer_id_nb_tx_7day_window
- terminal_id_avg_amount_15min_window
- terminal_id_avg_amount_30min_window
- terminal_id_avg_amount_60min_window
- terminal_id_nb_tx_14day_window
- terminal_id_nb_tx_15min_window
- terminal_id_nb_tx_1day_window
- terminal_id_nb_tx_30min_window
- terminal_id_nb_tx_60min_window
- terminal_id_nb_tx_7day_window
- terminal_id_risk_14day_window
- terminal_id_risk_1day_window
- terminal_id_risk_7day_window
- tx_amount
TARGET_COLUMN: tx_fraud
DATA_SCHEMA:
  timestamp: object
  tx_amount: float64
  tx_fraud: Int64
  entity_type_customer: Int64
  customer_id_nb_tx_1day_window: Int64
  customer_id_nb_tx_7day_window: Int64
  customer_id_nb_tx_14day_window: Int64
  customer_id_avg_amount_1day_window: float64
  customer_id_avg_amount_7day_window: float64
  customer_id_avg_amount_14day_window: float64
  customer_id_nb_tx_15min_window: Int64
  customer_id_avg_amount_15min_window: float64
  customer_id_nb_tx_30min_window: Int64
  customer_id_avg_amount_30min_window: float64
  customer_id_nb_tx_60min_window: Int64
  customer_id_avg_amount_60min_window: float64
  entity_type_terminal: Int64
  terminal_id_nb_tx_1day_window: Int64
  terminal_id_nb_tx_7day_window: Int64
  terminal_id_nb_tx_14day_window: Int64
  terminal_id_risk_1day_window: float64
  terminal_id_risk_7day_window: float64
  terminal_id_risk_14day_window: float64
  terminal_id_nb_tx_15min_window: Int64
  terminal_id_avg_amount_15min_window: float64
  terminal_id_nb_tx_30min_window: Int64
  terminal_id_avg_amount_30min_window: float64
  terminal_id_nb_tx_60min_window: Int64
  terminal_id_avg_amount_60min_window: float64
MODEL_NAME: ff_model
EXPERIMENT_NAME: "ff-experiment-{ID}"
DATA_URI: "gs://{PROJECT_ID}-fraudfinder/data"
TRAIN_DATA_URI: "gs://{PROJECT_ID}-fraudfinder/data/train"
READ_INSTANCES_TABLE: "ground_truth_{ID}"
READ_INSTANCES_URI: "bq://{PROJECT_ID}.tx.ground_truth_{ID}"
DATASET_NAME: "fraud_finder_dataset_{ID}"
JOB_NAME: "fraudfinder-train-xgb-{ID}"
ENDPOINT_NAME: ff_model_endpoint
MODEL_SERVING_IMAGE_URI: "us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-7:latest"
IMAGE_REPOSITORY: "fraudfinder-{ID}"
IMAGE_NAME: dask-xgb-classificator
IMAGE_TAG: latest
IMAGE_URI: "us-central1-docker.pkg.dev/{PROJECT_ID}/fraudfinder-{ID}/dask-xgb-classificator:latest"
TRAIN_COMPUTE: e2-standard-4
DEPLOY_COMPUTE: n1-standard-4
BASE_IMAGE: "python:3.10"
PIPELINE_NAME: "fraud-finder-xgb-pipeline-{ID}"
PIPELINE_ROOT: "gs://{PROJECT_ID}-fraudfinder/pipelines"
BQ_DATASET: tx
METRICS_URI: "gs://{PROJECT_ID}-fraudfinder/deliverables/metrics.json"
AVG_PR_THRESHOLD: 0.2
MODEL_THRESHOLD: 0.5
AVG_PR_CONDITION: avg_pr_condition
PERSISTENT_RESOURCE_ID: ai-takeoff
REPLICA_COUNT: 1
SERVICE_ACCOUNT: "{SERVICE_ACCOUNT}"
"""

In [29]:
!echo '{config}' | gsutil cp - gs://{BUCKET_NAME}/config/vertex_conf.yaml

Copying from <STDIN>...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              


In [25]:
print(config)


PROJECT_ID: fraud123-438914
BUCKET_NAME: fraud123-438914-fraudfinder
REGION: us-central1
ID: "fvde2"
CUSTOMER_ENTITY_ID: customer
CUSTOMER_ENTITY_ID_FIELD: customer_id
TERMINAL_ENTITY_ID: terminal
TERMINALS_ENTITY_ID_FIELD: terminal_id
FEATURESTORE_ID: "fraudfinder_fvde2"
FEATUREVIEW_ID: "fraudfinder_view_fvde2"
NETWORK: fraud-finder-network
SUBNET: https://www.googleapis.com/compute/v1/projects/fraud-finder-lab/regions/us-central1/subnetworks/us-central1
MODEL_REGISTRY: ff_model
RAW_BQ_TRANSACTION_TABLE_URI: "fraud123-438914.tx.tx"
RAW_BQ_LABELS_TABLE_URI: "fraud123-438914.tx.txlabels"
FEATURES_BQ_TABLE_URI: "fraud123-438914.tx.wide_features_table"
FEATURE_TIME: feature_ts
ONLINE_STORAGE_NODES: 1
SUBSCRIPTION_NAME: ff-tx-for-feat-eng-sub
SUBSCRIPTION_PATH: "projects/fraud123-438914/subscriptions/ff-tx-for-feat-eng-sub"
DROP_COLUMNS:
- timestamp
- entity_type_customer
- entity_type_terminal
FEAT_COLUMNS:
- customer_id_avg_amount_14day_window
- customer_id_avg_amount_15min_window
- cus

In [1]:
import yaml
from utils import gcs_read, VertexConfig


BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config_path = "config/vertex_conf.yaml"

with gcs_read(PROJECT_ID, BUCKET_NAME, "config/vertex_conf.yaml").open("r") as f:
    conf = yaml.safe_load(f)
vertex_config = VertexConfig(**conf)

!gcloud ai persistent-resources describe $PERSISTENT_RESOURCE_ID --project=$PROJECT_ID --region=$PERSISTENT_RESOURCE_REGION

ModuleNotFoundError: No module named 'utils'